## Recurrent Attention Model

In [2]:
import torch
import torch.nn as nn
from torchxai.base import XaiBase, XaiHook, XaiModel
from collections import OrderedDict
import matplotlib.pyplot as plt

In [ ]:
class GlimpseSensor(nn.Module):
    """Glimpse Sensor"""
    def __init__(self, g, k, s):
        """
        retina and location encoding
        ----
        The retina encoding `ρ(x, l)` extracts `k` square 
        patches centered at location `l`, with the first patch 
        being `g_w` × `g_w` pixels in size, and each successive 
        patch having twice the width of the previous. 
        The k patches are then all resized to `g_w × g_w` 
        and concatenated. 
        Glimpse locations `l` were encoded 
        as real-valued (x, y) coordinates with 
        (0, 0) being the center of the image x and 
        (−1, −1) being the top left corner of x.
        
        args:
        - g: size of first square patches
        - k: number of patches
        - s: scaling factor to control patches
        """
        super(GlimpseSensor, self).__init__()
        self.g = g
        self.k = k
        selk.s = s
    
    def extract_patch(self, x, l):
        B, C, H, W = x.size()
        
    
    def encode_coordinates(self, l):
        """
        encode coordinates to (-1, 1) range
        - center: (0, 0)
        - topleft: (-1, -1)
        """
        
        return 
        
    
    def forward(self, x, l):
        """
        args:
        - x: current time step input data, (B, C, H, W)
        - l: previous time step location, (B, 2)
        
        returns:
        - rho: retina-like representation
        """
        pass